In [1]:
# Make sure we have the new pandas 1.0 (Jan 2020) so we can use the new string dtype
! pip install pandas --upgrade

# Fuzzy text matching
! pip install "textdistance[extras]" --upgrade

Requirement already up-to-date: pandas in /usr/local/lib/python3.6/dist-packages (1.0.3)
     |████████████████████████████████| 51kB 2.8MB/s 
     |████████████████████████████████| 61kB 8.0MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 143kB 7.8MB/s 
     |████████████████████████████████| 890kB 48.1MB/s 
  Created wheel for pyxdameraulevenshtein: filename=pyxDamerauLevenshtein-1.5.3-cp36-cp36m-linux_x86_64.whl size=86797 sha256=7960214007cfbbf77df24a79b3832e6a2d0e60a788c026aa5a084208e418193d
  Stored in directory: /root/.cache/pip/wheels/37/66/2c/863e33407d169ea809ce2908e86556df205dcf5edc4bdd10a5
Successfully built pyxdameraulevenshtein
  Created wheel for python-levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144802 sha256=0be60c1ab999cb302742f89004f6b2647a373a923bbb00e21dbbc31422bfc14c
  Stored in directory: /root/.cache/pip/

In [0]:
import numpy as np
import pandas as pd
path = '/content/drive/My Drive/active/m5362_20sp_data_warehousing/PD4SDG/PD4SDG_newer/data'

def write_txt(df, fname, subdir='/output'):
    df.to_csv(f"{path}{subdir}/{fname}.txt", index=False, encoding='utf-8', sep='\t', header=True)

def write_csv(df, fname, subdir='/output'):
    df.to_csv(f"{path}{subdir}/{fname}.csv", index=False, encoding='utf-8', sep=',', header=True)

def read_txt(fname, subdir='/output'):
    return pd.read_csv(f"{path}{subdir}/{fname}.txt", encoding='utf-8', sep='\t', keep_default_na=False)

def read_csv(fname, subdir='/output'):
    return pd.read_csv(f"{path}{subdir}/{fname}.csv", encoding='utf-8', sep=',', keep_default_na=False)


pd.set_option('display.max_rows', None)
def disp(df, n=5):
    """Display n randomly selected rows of df"""
    n = min(n, df.shape[0])
    display(df.sample(n))

def string_cleaner(x, lev=1):
    y = x
    msg = ''
    if lev >= 1:
        try:
            y = (y.str.strip()                     ## remove leading and trailing white space
                .fillna('')                        ## fill nulls with ''
                .str.lower()                       ## lower case (temporary for replacement steps below)
                .str.replace(r'^.{0,1}$', '')      ## replace strings of length 0 or 1 with ''
                .str.replace('\r', '')             ## remove line breaks
                .str.replace('\n', '')             ## remove line breaks
            )
        except:
            msg += f"\n{x.name}: Level 1 cleaning fail"
        else:
            msg += f"\n{x.name}: Level 1 cleaning success"
            # if lev >= 2:
            #     try:
            #         y = (y.str.replace(r'^st\. ', 'saint ')    ## replace st. by saint
            #             .str.replace(r'^st '  , 'saint ')      ## replace st  by saint
            #             .str.replace(r' st\. ', ' saint ')    ## replace st. by saint
            #             .str.replace(r' st '  , ' saint ')      ## replace st  by saint
            #             .str.replace('&', 'and')           ## replace & by and
            #         )
            #     except:
            #         msg += f"\n{x.name}: Level 2 cleaning fail"
            #     else:
            #         msg += f"\n{x.name}: Level 2 cleaning success"
            #         # add more cleaning levels here if needed
    print(msg)
    return y

# Read & process PD4SDG

In [0]:
string_clean_level = 1

entity_name_fix = (pd.read_excel(f"{path}/input/raw/entity_name_fix_raw.xlsx")
    .apply(string_cleaner, lev=string_clean_level)
    # .sort_values(['orig', 'repl']).reset_index(drop=True)
)
write_txt(entity_name_fix, 'entity_name_fix', '/input')



entity_data = (pd.read_excel(f"{path}/input/raw/entity_data_raw.xlsx")
    .apply(string_cleaner, lev=string_clean_level)
    # .sort_values(['orig', 'repl']).reset_index(drop=True)
)
# df = (entity_data['site'].str.split(',', expand=True)
#     .rename(columns={0:'city_orig', 1:'admin1_orig', 2:'country_orig'})
# )
# idx = df['country_orig'].isna()
# df.loc[idx, 'country_orig'] = df.loc[idx, 'admin1_orig']
# idx = df['admin1_orig'] == df['country_orig']
# df.loc[idx, 'admin1_orig'] = ''


# entity_data = pd.concat([entity_data, df], axis=1)
# cols = ['id', 'name', 'type', 'country_orig', 'admin1_orig', 'city_orig', 'geonameid']
# entity_data = entity_data[cols]


write_txt(entity_data, 'entity_data', '/input')

disp(entity_data)


orig: Level 1 cleaning success

orig: Level 1 cleaning success

repl: Level 1 cleaning success

id: Level 1 cleaning fail

id: Level 1 cleaning fail

name: Level 1 cleaning success

type: Level 1 cleaning success

country_orig: Level 1 cleaning success

subcountry_orig: Level 1 cleaning success

city_orig: Level 1 cleaning success

geonameid: Level 1 cleaning fail


id  ... geonameid
392    393  ...   1185241
6886  6887  ...   4351335
7421  7422  ...   5128581
1142  1143  ...   2658660
4355  4356  ...   1853193

[5 rows x 7 columns]

In [0]:
pd4sdg = pd.read_excel(f"{path}/input/raw/PD4SDG.xlsx")

In [0]:
pd4sdg.columns = pd4sdg.columns.str.lower()
pd4sdg.rename(columns={'project location 1':'site', 'project_idx':'un_id'}, inplace=True)

pd4sdg['un_id'] = pd4sdg['un_id'].str.replace("'", "")
cl = {'title':1, 'site':1}
for col, lev in cl.items():
    pd4sdg[col] = string_cleaner(pd4sdg[col], lev=lev)


## Some rows have duplicate un_id.  Must uniquify.
f = lambda L: max(L, key=len)            ## returns longest string in list
project = (pd4sdg.groupby('un_id').agg(
        title = ('title', f),            ## select longest title in group
        site = ('site', f),              ## select longest site in group
        repeats = ('un_id', 'count'),
        )
    .reset_index()                       ## assign serial id
)
write_txt(project, 'project', '/input')



## Get project_entity links
## Make list of columns to stack
partners = ['lead partner'] + [f"partner {i+1}" for i in range(237)]
L = []
for (n, col) in enumerate(partners):
    df = pd4sdg[['un_id', col]]      ## get un_id and one partner column
    df[col] = string_cleaner(df[col], lev=string_clean_level)
    idx = df[col] != ''              ## find empty partner names
    df = df[idx]                     ## remove empty partner names
    df.columns = ['un_id' , 'name']  ## rename columns
    df['n'] = n                      ## record which partner number this was
    L.append(df)                     ## append to list L

project_entity = pd.concat(L)        ## concatenate the lists stored in L
write_txt(project_entity, 'project_entity', '/input')

# disp(pd4sdg)
# disp(project)
# disp(project_entity)


title: Level 1 cleaning success

site: Level 1 cleaning success

lead partner: Level 1 cleaning success

partner 1: Level 1 cleaning success

partner 2: Level 1 cleaning success

partner 3: Level 1 cleaning success

partner 4: Level 1 cleaning success

partner 5: Level 1 cleaning success

partner 6: Level 1 cleaning success

partner 7: Level 1 cleaning success

partner 8: Level 1 cleaning success

partner 9: Level 1 cleaning success

partner 10: Level 1 cleaning success

partner 11: Level 1 cleaning success

partner 12: Level 1 cleaning success

partner 13: Level 1 cleaning success

partner 14: Level 1 cleaning success

partner 15: Level 1 cleaning success

partner 16: Level 1 cleaning success


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



partner 17: Level 1 cleaning success

partner 18: Level 1 cleaning success

partner 19: Level 1 cleaning success

partner 20: Level 1 cleaning success

partner 21: Level 1 cleaning success

partner 22: Level 1 cleaning success

partner 23: Level 1 cleaning success

partner 24: Level 1 cleaning success

partner 25: Level 1 cleaning success

partner 26: Level 1 cleaning success

partner 27: Level 1 cleaning success

partner 28: Level 1 cleaning success

partner 29: Level 1 cleaning success

partner 30: Level 1 cleaning success

partner 31: Level 1 cleaning success

partner 32: Level 1 cleaning success

partner 33: Level 1 cleaning success

partner 34: Level 1 cleaning success

partner 35: Level 1 cleaning success

partner 36: Level 1 cleaning success

partner 37: Level 1 cleaning success

partner 38: Level 1 cleaning success

partner 39: Level 1 cleaning success

partner 40: Level 1 cleaning success

partner 41: Level 1 cleaning success

partner 42: Level 1 cleaning success

partner 43:

# Fuzzy Text Matching

In [0]:
### improved

import textdistance
def textdist_func(a, b):
    try:
        a[0].lower() + b[0].lower()
    except:
        return 0.0
    else:
        return textdistance.levenshtein.normalized_similarity(a, b)
textdist_ufunc = np.frompyfunc(textdist_func, 2, 1)
def textdist(A, B):
    return textdist_ufunc.outer(A, B).astype('float')


def large(X, k=2, axis=0):
    if k <= 1:
        srt = np.argmax(X, axis)  # find
        srt = np.expand_dims(srt, axis)
        X = np.take_along_axis(X, srt, axis)
    else:
        X = np.rollaxis(X, axis, 0)
        srt1 = np.argpartition(X, -k, axis=0)[-k:]
        X = np.take_along_axis(X, srt1, axis=0)
        srt2 = np.argsort(X, axis=0)[::-1]
        X = np.take_along_axis(X, srt2, axis=0)
        srt = np.take_along_axis(srt1, srt2, axis=0)
        X   = np.rollaxis(X, 0, axis+1)
        srt = np.rollaxis(srt, 0, axis+1)
    return X, srt


def text_match(orig, targ, num_matches=1):
    def f(X):
        d = dict()
        d['df'] = pd.DataFrame(X)
        d['unique'] = d['df'].drop_duplicates()
        d['index'] = d['unique'].index.to_numpy()
        d['names'] = d['unique'].iloc[:, 0].to_numpy()
        d['data'] = d['unique'].iloc[:, 1:]
        return d
    orig = f(orig)
    targ = f(targ)
    num_matches = min(num_matches, len(targ['unique']))
    scores = textdist(orig['names'], targ['names'])
    best_score, srt = large(scores.T, k=num_matches, axis=0)

    try:
        index = targ['df'].index.name
    except:
        index = 'index'
    try:
        name = targ['df'].columns[0]
    except:
        name = 'name'
    
    df = pd.DataFrame(index=orig['names'])
    for i in range(num_matches):
        df[  f"score_{i+1}"] = (best_score[i]*100).round(1)
        df[f"{index}_{i+1}"] = targ['index'][srt[i]]
        df[   f"{name}_{i+1}"] = targ['names'][srt[i]]
        for col, dat in targ['data'].iteritems():
            df[f"{col}_{i+1}"] = dat.values[srt[i]]

    new_df = orig['df']
    join_col = new_df.columns[0]
    new_df = new_df.join(df, how='left', on=join_col)
   
    return new_df, scores

In [0]:
# %%time
num_matches = 2

entity_data_full = read_csv('entity_data_full', '/output').set_index('entity_id')
cols = ['entity_name', 'entity_type', 'city_name', 'country_name', 'num_projects']
orig = entity_data_full[cols]
targ = orig.copy()

# orig = orig.iloc[:10]
entity_similarity, s = text_match(orig, targ, num_matches)

cols = ['score', 'entity_id', 'entity_name', 'entity_type', 'city_name', 'country_name', 'num_projects']
idx = entity_similarity['entity_id_1'] == entity_similarity.index
for c in cols:
    w1, w2 = f"{c}_1", f"{c}_2"
    entity_similarity.loc[idx, w1] = entity_similarity.loc[idx, w2]
    entity_similarity.drop(columns=w2, inplace=True)

entity_similarity = entity_similarity.sort_values(by=['score_1', 'entity_name'], ascending=[False, True]).reset_index()

idx = (
    (entity_similarity['score_1'] > 95)
    & (entity_similarity['country_name'] == entity_similarity['country_name_1'])
    & (
        (entity_similarity['num_projects'] < entity_similarity['num_projects_1'])
        | (
            (entity_similarity['num_projects'] == entity_similarity['num_projects_1'])
            & (entity_similarity['entity_name'] > entity_similarity['entity_name_1'])
        )
    )
)
cols = ['entity_id', 'entity_name', 'entity_type', 'city_name', 'country_name', 'num_projects']
for c in cols:
    w1 = f"{c}_1"
    w2 = f"{c}_repl"
    entity_similarity[w2] = entity_similarity[c]
    entity_similarity.loc[idx, w2] = entity_similarity.loc[idx, w1]

    # entity_similarity[w2] = entity_similarity[c]
    # entity_similarity.loc[~idx, w2] = entity_similarity.loc[~idx, w1]




# entity_similarity['new_entity_id'] = entity_similarity['entity_id']
# idx = (entity_similarity['score_repl'] > 95) & (entity_similarity['country_name'] == entity_similarity['country_name_repl'])
# entity_similarity.loc[idx, 'new_entity_id'] = entity_similarity.loc[idx, 'entity_id_repl']


write_csv(entity_similarity, 'entity_similarity', '/output')
entity_similarity.head()

entity_id  ... num_projects_repl
0       5666  ...                 3
1       5649  ...                 3
2       4325  ...                 3
3       4326  ...                 3
4       4629  ...                 8

[5 rows x 19 columns]

In [0]:
entity_similarity = read_csv('entity_similarity', '/output')
entity_similarity.head()

entity_id  ... num_projects_repl
0          9  ...                 1
1          6  ...                13
2         10  ...                 1
3          2  ...                 1
4          5  ...                 1

[5 rows x 13 columns]

In [0]:
# entity_similarity['replace_entity_id'] = entity_similarity['replace_entity_id']

In [0]:
entity_similarity = read_csv('entity_similarity', '/output')

entity_similarity['new_entity_id'] = entity_similarity['entity_id']
idx = (entity_similarity['score_repl'] > 95) & (entity_similarity['country_name'] == entity_similarity['country_name_repl'])
entity_similarity.loc[idx, 'new_entity_id'] = entity_similarity.loc[idx, 'entity_id_repl']



# entity_similarity = entity_similarity.iloc[:10]
# entity_similarity.head()



entity_similarity.groupby(['score_repl', 'country_name'])
cols = ['entity_id', 'entity_name', 'entity_type', 'city_name', 'country_name', 'num_projects']
def f(x):
    y = x.sort_values(by=['num_projects', 'entity_name'], ascending=[False, True])
    z = y[cols].iloc[0,:]
    return x.loc[i, 'entity_id']
entity_similarity.apply(f)

In [4]:
num_matches = 3

world = read_txt('world', '/output').set_index('city_geonameid')
entity_data_full = read_csv('entity_data_full', '/output')
entity_data = entity_data_full[['entity_name', 'entity_type', 'country_orig', 'subcountry_orig', 'city_orig', 'country_code', 'city_geonameid']]
# entity_data = entity_data.iloc[:10]

entity_grp = entity_data.groupby('country_code')
world_grp  = world.groupby('country_code')

L = []
for cc, ent in entity_grp:
    print(f"Doing {cc}")
    orig = ent['city_orig']
    g = world_grp.get_group(cc)
    targ = g[['city_name', 'admin1_name']]
    df, s = text_match(orig, targ, num_matches)
    # disp(df)
    df.drop(columns='city_orig', inplace=True)
    L.append(df)

matches = pd.concat(L)
entity_data_matches = entity_data.join(matches, how='left')
entity_data_matches['city_geonameid'] = entity_data_matches['city_geonameid_1']
entity_data_matches.sort_values(by=['country_code', 'score_1', 'city_orig'], ascending=[True, False, True], inplace=True)

display(entity_data_matches.head())
write_csv(entity_data_matches, 'entity_data_matches', '/output')

Doing ad
Doing ae
Doing af
Doing ag
Doing al
Doing am
Doing ao
Doing ar
Doing at
Doing au
Doing aw
Doing az
Doing ba
Doing bb
Doing bd
Doing be
Doing bf
Doing bg
Doing bh
Doing bi
Doing bj
Doing bm
Doing bn
Doing bo
Doing br
Doing bs
Doing bt
Doing bw
Doing by
Doing bz
Doing ca
Doing cd
Doing cf
Doing cg
Doing ch
Doing ci
Doing ck
Doing cl
Doing cm
Doing cn
Doing co
Doing cr
Doing cu
Doing cv
Doing cw
Doing cy
Doing cz
Doing de
Doing dj
Doing dk
Doing dm
Doing do
Doing dz
Doing ec
Doing ee
Doing eg
Doing er
Doing es
Doing et
Doing fi
Doing fj
Doing fm
Doing fo
Doing fr
Doing ga
Doing gb
Doing gd
Doing ge
Doing gh
Doing gm
Doing gn
Doing gq
Doing gr
Doing gt
Doing gu
Doing gw
Doing gy
Doing hn
Doing hr
Doing ht
Doing hu
Doing id
Doing ie
Doing il
Doing in
Doing iq
Doing ir
Doing is
Doing it
Doing jm
Doing jo
Doing jp
Doing ke
Doing kg
Doing kh
Doing ki
Doing km
Doing kn
Doing kp
Doing kr
Doing kw
Doing ky
Doing kz
Doing la
Doing lb
Doing lc
Doing li
Doing lk
Doing lr
Doing ls
Doing lt
D

entity_name  ...        admin1_name_3
0                         andorra  ...  sant julià de loria
1  abu dhabi fund for development  ...                dubai
2                         borouge  ...                dubai
3    environment agency abu dhabi  ...                dubai
4                    eye on earth  ...                dubai

[5 rows x 19 columns]

In [8]:
disp(entity_data_matches)

entity_name  ...       admin1_name_3
6645                industrial union of donbass  ...            mykolaiv
2796  national school of mines of saint-etienne  ...  nouvelle-aquitaine
330                                  azerbaijan  ...               qusar
5860   foundation for communication initiatives  ...              caraga
5136                academy of learning nigeria  ...              jigawa

[5 rows x 19 columns]

# old

In [0]:
### old

num_matches = 3

world = read_txt('world', '/output').set_index('geonameid')
entity_data = read_txt('entity_data', '/output')

entity_data_full = read_csv('entity_data_full', '/output')

entity_data = entity_data_full[['entity_name', 'entity_type', 'country_orig', 'subcountry_orig', 'city_orig', 'country_code']]



# country = world[['country_name', 'country_code']].drop_duplicates().set_index('country_name')
# entity_country = entity_data.join(country, how='left', on='country_orig').set_index('id')
idx = entity_country['country_code'].isna()
entity_country_nomatch = entity_country[idx].sort_values(['country_orig', 'city_orig', 'name'])
entity_country_matches = entity_country[~idx]

entity_country_matches = entity_country_matches.iloc[:10]

entity_grp = entity_country_matches.groupby('country_code')
world_grp  = world.groupby('country_code')

L = []
for cc, ent in entity_grp:
    print(f"Doing {cc}")
    orig = ent['city_orig']
    g = world_grp.get_group(cc)
    targ = g[['name', 'admin1_name']]
    df, s = text_match(orig, targ, num_matches)
    # disp(df)
    df.drop(columns='city_orig', inplace=True)
    L.append(df)

matches = pd.concat(L)
entity_city_matches = entity_country_matches.join(matches, how='left')
entity_city_matches['geonameid'] = entity_city_matches['geonameid_1']
entity_city_matches.sort_values(by=['country_code', 'score_1', 'city_orig'], ascending=[True, False, True], inplace=True)

entity_data_matches = pd.concat([entity_country_nomatch, entity_city_matches]).fillna('').convert_dtypes().reset_index()
display(entity_data_matches.head())

write_csv(entity_data_matches, 'entity_data_matches', '/output')

Doing ad
Doing ae


id                            name  ...               name_3        admin1_name_3
0   1                         andorra  ...  sant julià de lòria  sant julià de loria
1   2  abu dhabi fund for development  ...                dubai                dubai
2   3                         borouge  ...                dubai                dubai
3   4    environment agency abu dhabi  ...                dubai                dubai
4   5                    eye on earth  ...                dubai                dubai

[5 rows x 20 columns]

In [0]:
### old

import textdistance
def textdist_func(a, b):
    try:
        a[0].lower() + b[0].lower()
    except:
        return 0.0
    else:
        return textdistance.levenshtein.normalized_similarity(a, b)
textdist_ufunc = np.frompyfunc(textdist_func, 2, 1)
def textdist(A, B):
    return textdist_ufunc.outer(A, B).astype('float')


def large(X, k=2, axis=0):
    if k <= 1:
        srt = np.argmax(X, axis)  # find
        srt = np.expand_dims(srt, axis)
        X = np.take_along_axis(X, srt, axis)
    else:
        X = np.rollaxis(X, axis, 0)
        srt1 = np.argpartition(X, -k, axis=0)[-k:]
        X = np.take_along_axis(X, srt1, axis=0)
        srt2 = np.argsort(X, axis=0)[::-1]
        X = np.take_along_axis(X, srt2, axis=0)
        srt = np.take_along_axis(srt1, srt2, axis=0)
        X   = np.rollaxis(X, 0, axis+1)
        srt = np.rollaxis(srt, 0, axis+1)
    return X, srt


def text_match(orig, targ, num_matches=1):
    orig_unique = np.unique(np.ravel(orig))
    
    targ = pd.DataFrame(targ)
    try:
        label = targ.index.name
    except:
        label = 'index'
    targ_unique = targ.drop_duplicates()
    num_matches = min(num_matches, len(targ_unique))
    
    index = targ_unique.index.to_numpy()
    names = targ_unique.iloc[:, 0].to_numpy()
    data  = targ_unique.iloc[:, 1:]

    scores = textdist(orig_unique, names)
    best_score, srt = large(scores.T, k=num_matches, axis=0)

    df = pd.DataFrame(index=orig_unique)
    for i in range(num_matches):
        df[  f"score_{i+1}"] = (best_score[i]*100).round(1)
        df[f"{label}_{i+1}"] = index[srt[i]]
        df[   f"name_{i+1}"] = names[srt[i]]
        for col, dat in data.iteritems():
            df[f"{col}_{i+1}"] = dat.values[srt[i]]

    new_df = pd.DataFrame()
    new_df['orig'] = orig
    new_df = new_df.join(df, how='left', on='orig')
   
    return new_df, scores